In [1]:
import gym
import torch
import numpy as np
import sys
from torchvision.transforms import ToTensor, Lambda
from torch import nn
from my_model import DQN
from utils import dataset, dataloader, buffer
import trainer
import random
import torch.onnx as onnx
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('RLresult/cartpole/dqn')
device = 'cuda' if torch.cuda.is_available() else 'cpu'


cuda


In [2]:
env = gym.make('CartPole-v1') 
print("act", env.action_space)
print("obs", env.observation_space)


act Discrete(2)
obs Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [3]:
HIDDEN_SIZE = 32
BATCH_SIZE = 10000
learning_rate = 0.01
CART_ACTION = 2
CAPACITY = 10000
TRAIN_ITER = CAPACITY*10/BATCH_SIZE
PARAM_PATH = 'DQN_param/cartpoleDQN.pth'
PARAM_PATH_TEST = 'DQN_param/cartpoleDQN_test.pth'
#print(Transition(2,1,2,3))
#Transition(state=2, action=1, next_state=2, reward=3)
    

In [12]:
%%time

input_state_size = len(env.observation_space.sample())

updatedDQN = DQN.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
baseDQN = DQN.DQN(input_state_size, HIDDEN_SIZE, CART_ACTION).to(device)
mybuffer = buffer.ForwardMem(baseDQN, env, "cart")


updatedDQN.load_state_dict(torch.load(PARAM_PATH_TEST))
baseDQN.load_state_dict(updatedDQN.state_dict())
baseDQN.eval()


#for p in updatedDQN.parameters(): 
#    print(p)

mycartdata = datset.EnvData(capacity = CAPACITY)
renewal_memory(CAPACITY, rendering = False)
dqn_dataloader = custom_dataloder(mycartdata, batch_size=BATCH_SIZE)
mytrain = trainer.Train("cart", dqn_dataloader, updatedDQN, baseDQN)

load_memory_complete
CPU times: user 3.27 s, sys: 769 µs, total: 3.27 s
Wall time: 3.27 s


In [13]:
optimizer = torch.optim.SGD(updatedDQN.parameters(), lr=learning_rate)
train_iter = 100
i = 0
while(i < train_iter):
    i = i + 1
    loss = mytrain.training(TRAIN_ITER, BATCH_SIZE, optimizer)
    writer.add_scalar("loss", loss, i)
    torch.save(updatedDQN.state_dict(), PARAM_PATH_TEST)
    baseDQN.load_state_dict(updatedDQN.state_dict())
    baseDQN.eval()
    mybuffer.renewal_memory(CAPACITY, rendering = False)
    
env.close()


meanloss =  tensor(1.8030, device='cuda:0', grad_fn=<DivBackward0>)
load_memory_complete
meanloss =  tensor(1.8015, device='cuda:0', grad_fn=<DivBackward0>)
load_memory_complete


KeyboardInterrupt: 

In [ ]:
writer.flush()
writer.close()

In [18]:
renewal_memory(CAPACITY, rendering = True)

/root/anaconda3/lib/python3.6/site-packages/gym/logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


KeyboardInterrupt: 

In [ ]:
#for p in updatedDQN.parameters(): 
#    print("params = ", p)

In [ ]:
'''
loss = nn.MSELoss()
input1 = torch.tensor(np.array([[-4,-5]], dtype=np.float32), requires_grad=True)
input2 = torch.tensor(np.array([[2,1]], dtype=np.float32), requires_grad=True)
input3 = torch.tensor(np.array([[-6,-2]], dtype=np.float32), requires_grad=True)
input4 = torch.cat((input1, input2),axis = 0)
input4 = torch.cat((input4, input3),axis = 0)
print(input4)
target = torch.tensor(np.array([[1, 1]], dtype = np.float32))
target1 = torch.cat((target, target), axis = 0)
target = torch.cat((target, target1), axis = 0)
print(target)
output = loss(input4, target)
print(output)
'''

In [ ]:
'''
CART_ACTION = 3
class DQN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(DQN, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, CART_ACTION),
            nn.ReLU()
        )

        
    def forward(self, input):
        output = self.linear_relu_stack(input)
        return output
    
testDQN = DQN(2,2)
'''